### Ranking Replication
* Zweck: Replikation des Kapitel "Ranking" aus dem Paper "Predicting Vulnerable Software Components"
* Matrix: Aktueller Stand "current" des mozilla-central Repository
* Features: includes
* Modell: Support Vector Regressionsmodell

#### Setup
* Training-Set/Test-Set: Stratified sampling auf einer Matrix (2/3 : 1/3)

#### Results
Angenommen man konzentriert seine Ressourcen auf die Optimierung der 1% verwundbarsten Komponenten. Bei der Auswahl verlässt man sich auf die Vorhersage des Regressionsmodells. Dieses Skript berechnet den Faktor zwischen der Anzahl Verwundbarkeiten die in diesem Fall behoben werden und der Anzahl Verwundbarkeiten die bei der Konzentration auf 1% maximal behoben werden können.
* Resultat: Faktor zwischen behobenen und maximal behebbaren Verwundbarkeiten.

In [43]:
from imports.matrix_helper import MatrixHelper
from imports.prediction_helper import PredictionHelper
from scipy.stats import rankdata
from scipy.stats import spearmanr
import numpy as np

matrix_helper = MatrixHelper()
matrices = matrix_helper.load_from_parse('data/matrices/matrix_reg_incl_current.pickle')

# Instantiate Prediction Helper Class and predict values for compare matrix
prediction_helper = PredictionHelper()
prediction_helper.calculate_validation_compare_matrix(matrices, sampling_factor=(2.0/3), prediction_type='SVR')

# get the compare matrix twice:
# Once sorted according to the PREDICTED number of vulnerabilities
# Once sorted according to the ACTUAL number of vulnerabilities
compare_matrix_sorted_predicted = prediction_helper.get_compare_matrix_sorted()
compare_matrix_sorted_actual = prediction_helper.get_compare_matrix_sorted(reference_column=2)

# Crop first 1% of both matrices
relevant_samples_count = int(round(0.01 * len(compare_matrix_sorted_predicted)))
compare_matrix_sorted_predicted = compare_matrix_sorted_predicted[range(relevant_samples_count)]
compare_matrix_sorted_actual = compare_matrix_sorted_actual[range(relevant_samples_count)]

# Sum up the actual number of vulnerabilities in both matrices
fixed_predicted = sum(np.array(compare_matrix_sorted_predicted[:, 2], dtype='f'))
fixed_actual = sum(np.array(compare_matrix_sorted_actual[:, 2], dtype='f'))

# Calculate the prediction factor
print('Factor between the number of fixed components due to the prediction and the maximum number of fixed components:') 
print('Factor: {}'.format(fixed_predicted / fixed_actual))

Factor between the number of fixed components due to the prediction and the maximum number of fixed components:
Factor: 0.794069192751
